This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-model-deploy-and-serve/pytorch-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: Pytorch Single Input Output

The following tutorial demonstrates how to upload a Pytorch Single Input Output model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a Pytorch Single Input Output to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'pytorch-single-io'
pipeline_name = 'pytorch-single-io'

model_name = 'pytorch-single-io'
model_file_name = "./models/model-auto-conversion_pytorch_single_io_model.pt"

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for PyTorch models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a PyTorch model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, PyTorch model Required*) | Set as the `Framework.PyTorch`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The input schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The output schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
| `convert_wait` | `bool` (*Upload Method Optional, PyTorch model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float32(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float32(), list_size=1))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.PYTORCH`.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name,
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime..
Model is attempting loading to a native runtime.successful

Ready


Name,pytorch-single-io
Version,61bda7ba-1c11-400d-bec0-5f59a7df774e
File Name,model-auto-conversion_pytorch_single_io_model.pt
SHA,23bdbafc51c3df7ac84e5f8b2833c592d7da2b27715a7da3e45bf732ea85b8bb
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Jul 21:35:17
Workspace id,160
Workspace name,pytorch-single-io


In [7]:
model.config().runtime()

'onnx'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
# clear the pipeline if it was used before
pipeline.undeploy()
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)

Waiting for deployment - this will take up to 45s ............ ok


name,pytorch-single-io
created,2024-07-22 21:35:04.206882+00:00
last_updated,2024-07-22 21:35:20.298724+00:00
deployed,True
workspace_id,160
workspace_name,pytorch-single-io
arch,x86
accel,none
tags,
versions,"fe95a886-1a73-49c5-99f8-0377352cf4af, f9d6ec22-d548-46bc-9ef1-cbb21d981131"
steps,pytorch-single-io


### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [10]:
mock_inference_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({"input": mock_inference_data.tolist()})
pipeline.infer(mock_dataframe)

,time,in.input,out.output,anomaly.count
0,2024-07-22 21:35:34.043,"[0.725922699, 0.7791820961, 0.2610608571, 0.58...",[-0.06809704],0
1,2024-07-22 21:35:34.043,"[0.8672469691, 0.6290836987, 0.2015037383, 0.3...",[-0.14699489],0
2,2024-07-22 21:35:34.043,"[0.0662654846, 0.9508379362, 0.9278146334, 0.1...",[0.09484814],0
3,2024-07-22 21:35:34.043,"[0.2694985706, 0.2832842058, 0.7563686475, 0.2...",[-0.065485604],0
4,2024-07-22 21:35:34.043,"[0.8520149523, 0.2359237792, 0.9577675634, 0.7...",[-0.05361571],0
5,2024-07-22 21:35:34.043,"[0.6215170133, 0.1841670051, 0.8056805576, 0.4...",[0.075670704],0
6,2024-07-22 21:35:34.043,"[0.3945733745, 0.2095386971, 0.4295028084, 0.9...",[-0.11618956],0
7,2024-07-22 21:35:34.043,"[0.4549542909, 0.5437075564, 0.0097600902, 0.3...",[-0.050185442],0
8,2024-07-22 21:35:34.043,"[0.0067956206, 0.0565600285, 0.8498016037, 0.0...",[0.21018298],0
9,2024-07-22 21:35:34.043,"[0.9427696818, 0.6637687735, 0.2103963512, 0.5...",[-0.14513397],0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [12]:
pipeline.undeploy()

 ok


name,pytorch-single-io
created,2024-07-22 21:35:04.206882+00:00
last_updated,2024-07-22 21:35:20.298724+00:00
deployed,False
workspace_id,160
workspace_name,pytorch-single-io
arch,x86
accel,none
tags,
versions,"fe95a886-1a73-49c5-99f8-0377352cf4af, f9d6ec22-d548-46bc-9ef1-cbb21d981131"
steps,pytorch-single-io
